# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix, precision_score,\
recall_score,accuracy_score,  f1_score,  make_scorer
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk import word_tokenize

import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * from messages", engine)
df=df.drop(['original'], axis=1)
df2 =df.dropna()
X = df2['message']
col_names=df2.drop(['id','message','genre'], axis=1).columns
Y =df2[col_names]

In [3]:
Y.shape

(26207, 36)

In [4]:
Y.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

In [5]:
X.shape

(26207,)

### 2. Write a tokenization function to process your text data

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [7]:
def replace_urls(text):
    """
    Outputs editted version of the input Python str object `text` 
    replacing all urls in text with str 'urlplaceholder'.
    
    INPUT:
        - text - Python str object - a raw text data
        
    OUTPUT:
        - text - Python str object - An editted version of the input data `text` 
          with all urls in text replacing with str 'urlplaceholder'.
    """
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
        
    return text

In [8]:
def tokenize(text):
    """
      Takes a Python string object and outputs list of processed words 
       of the text.
      INPUT:
        - text - Python str object - A raw text data
      OUTPUT:
        - stem_words - Python list object - list of processed words using the input `text`.
     """
    #function with raw text data    
    text = replace_urls(text)
    # Removes punctuations and covert to lower case 
    #text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower() 
    text = word_tokenize(text) 
    # Remove stop words
    text = [w for w in text if w not in stopwords.words("english")]
    # Lemmatize verbs by specifying pos
    text = [WordNetLemmatizer().lemmatize(w) for w in text]
    
    return text

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fcc28407620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_state=None

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=49)
X_train.count()

19655

In [12]:
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer() 
clf = RandomForestClassifier()  # MultinomialNB()

In [13]:
y_train.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

In [14]:
pipeline.fit(X_train, y_train)
#tfidf.fit_transform(vect.fit_transform(X_test))
pipeline = Pipeline([
         ('vect', CountVectorizer(tokenizer=tokenize)),
         ('tfidf', TfidfTransformer()),
         ('clf', RandomForestClassifier())])

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

In [15]:
y_pred = pipeline.predict(X_test)

In [16]:
y_pred.shape,y_test.shape,len(list(Y.columns))

((6552, 36), (6552, 36), 36)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
accuracy = (y_pred == y_test).mean()

print("Accuracy:", accuracy)

Accuracy: related                   0.709860
request                   0.828907
offer                     0.997100
aid_related               0.567155
medical_help              0.919567
medical_products          0.945971
search_and_rescue         0.970543
security                  0.980006
military                  0.968254
child_alone               1.000000
water                     0.932082
food                      0.889347
shelter                   0.909799
clothing                  0.983974
money                     0.975427
missing_people            0.988095
refugees                  0.964896
death                     0.953144
other_aid                 0.865385
infrastructure_related    0.933455
transport                 0.956197
buildings                 0.947344
electricity               0.981380
tools                     0.993437
hospitals                 0.988553
shops                     0.996337
aid_centers               0.989621
other_infrastructure      0.954365
weather_re

In [18]:
print(classification_report(y_test,y_pred,target_names = df.columns[3:]))

                        precision    recall  f1-score   support

               related       0.77      0.88      0.82      5002
               request       0.48      0.11      0.17      1113
                 offer       0.00      0.00      0.00        19
           aid_related       0.45      0.18      0.25      2722
          medical_help       0.11      0.00      0.00       520
      medical_products       0.17      0.01      0.01       346
     search_and_rescue       0.33      0.01      0.01       192
              security       0.00      0.00      0.00       126
              military       0.00      0.00      0.00       207
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       438
                  food       0.43      0.02      0.03       721
               shelter       0.26      0.01      0.02       582
              clothing       0.00      0.00      0.00       103
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__bootstrap', 'clf__class_weight', 'clf__criterion', 'clf__max_depth', 'clf__max_features', 'clf__max_leaf_nodes', 'clf__min_impurity_decrease', 'clf__min_impurity_split', 'clf__min_samples_leaf', 'clf__min_samples_split', 'clf__min_weight_fraction_leaf', 'clf__n_estimators', 'clf__n_jobs', 'clf__oob_score', 'clf__random_state', 'clf__verbose', 'clf__warm_start'])

In [20]:
parameters = {
    'clf__estimator__n_estimators': [10, 100],
     'clf__max_depth': [5, 40],
    'estimator__criterion':['entropy','gini']
    }
cv = GridSearchCV(
    pipeline, 
    param_grid = parameters,
    cv=3,
    scoring='accuracy'
    )

cv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__n_estimators': [10, 100], 'clf__max_depth': [5, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
new_model = cv.fit(X_train,y_train)

In [21]:
y_pred =cv.predict(X_test)
print(classification_report(y_test,y_pred,target_names = df.columns[3:]))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      5002
               request       0.00      0.00      0.00      1113
                 offer       0.00      0.00      0.00        19
           aid_related       0.55      0.00      0.00      2722
          medical_help       0.00      0.00      0.00       520
      medical_products       0.00      0.00      0.00       346
     search_and_rescue       0.00      0.00      0.00       192
              security       0.00      0.00      0.00       126
              military       0.00      0.00      0.00       207
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       438
                  food       0.00      0.00      0.00       721
               shelter       0.00      0.00      0.00       582
              clothing       0.00      0.00      0.00       103
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
filename = 'random_forest_classifier_model_precision.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.